In [40]:
'''
Notes:

Dense - A regular layer of neurons in a neural network.
Loss - It’s a method of evaluating how well your algorithm models your dataset. If your predictions are totally off, your loss function will output a higher number. If they’re pretty good, it’ll output a lower number.
Optimizer - Learning rate
Metrics - What parameters are being used to evaluate the overall quality of the ML

By age
'''

'\nNotes:\n\nDense - A regular layer of neurons in a neural network.\nLoss - It’s a method of evaluating how well your algorithm models your dataset. If your predictions are totally off, your loss function will output a higher number. If they’re pretty good, it’ll output a lower number.\nOptimizer - Learning rate\nMetrics - What parameters are being used to evaluate the overall quality of the ML\n\n'

In [41]:
import itertools
import os
import math
import numpy as np
import pandas as pd
import tensorflow as tf
import pickle
import graphviz

from IPython.display import Image
from IPython.core.display import HTML
from sklearn import tree
from sklearn.preprocessing import LabelEncoder
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.utils.vis_utils import plot_model
layers = keras.layers

print("Importing done")

Importing done


In [42]:
URL = "https://github.com/Brunel-Visualization/Brunel/raw/master/python/examples/data/whiskey.csv"
path = tf.keras.utils.get_file(URL.split('/')[-1], URL)

print("Data fetching done")

Data fetching done


In [43]:
# Convert the data to a Pandas data frame
data = pd.read_csv(path)

# Shuffle the data
data = data.sample(frac=1)

features = list(data.columns.values)
print(features)

print("Data reading done")

['Name', 'Rating', 'Country', 'Category', 'Price', 'ABV', 'Age', 'Brand']
Data reading done


In [44]:
# Do some preprocessing to limit the # of wine varities in the dataset
data = data[pd.notnull(data['Name'])]
data = data[pd.notnull(data['Category'])]
data = data[pd.notnull(data['Price'])]
data = data[pd.notnull(data['Rating'])]
data = data[pd.notnull(data['Age'])]

print("Data processing done")

Data processing done


In [45]:
# Split data into train and test
train_size = int(len(data) * .8)
print ("Total size: %d" % (len(data)))
print ("Train size: %d" % train_size)
print ("Test size: %d" % (len(data) - train_size))

print("Data splitting done")

Total size: 164
Train size: 131
Test size: 33
Data splitting done


In [46]:
# Train features
category_train = data['Category'][:train_size]
name_train = data['Name'][:train_size]

# Train labels
labels_trainOne = data['Price'][:train_size]
labels_trainTwo = data['Rating'][:train_size]

# Test features
category_test = data['Category'][:train_size]
name_test = data['Name'][:train_size]

# Test labels
labels_testOne = data['Price'][:train_size]
labels_testTwo = data['Rating'][:train_size]

print("Labeling done")

Labeling done


In [47]:
# Create a tokenizer to preprocess our text descriptions
vocab_size = 10 # This is a hyperparameter, experiment with different values for your dataset
tokenize = keras.preprocessing.text.Tokenizer(num_words=vocab_size, char_level=False)
tokenize.fit_on_texts(name_train) # only fit on train

print("Tokeninzing done")

Tokeninzing done


In [48]:
# Wide feature 1: sparse bag of words (bow) vocab_size vector
name_bow_train = tokenize.texts_to_matrix(name_train)
name_bow_test = tokenize.texts_to_matrix(name_test)

In [49]:
# Wide feature 2: one-hot vector of variety categories

# Use sklearn utility to convert label strings to numbered index
encoder = LabelEncoder()
encoder.fit(category_train)
category_train = encoder.transform(category_train)
category_test = encoder.transform(category_test)
num_classes = np.max(category_train) + 1

# Convert labels to one hot
category_train = keras.utils.to_categorical(category_train, num_classes)
category_test = keras.utils.to_categorical(category_test, num_classes)

In [50]:
# Define our wide model with the functional API
bow_inputs = layers.Input(shape=(vocab_size,))
variety_inputs = layers.Input(shape=(num_classes,))
merged_layer = layers.concatenate([bow_inputs, variety_inputs])
merged_layer = layers.Dense(256, activation='relu')(merged_layer)
predictions = layers.Dense(1)(merged_layer)
wide_model = keras.Model(inputs=[bow_inputs, variety_inputs], outputs=predictions)
wide_model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])
plot_model(wide_model, to_file='model_plotA.png', show_shapes=True, show_layer_names=True)

Image(url = "model_plotA.png", width = 500, height = 500)

In [51]:
# Deep model feature: word embeddings of wine descriptions
train_embed = tokenize.texts_to_sequences(name_train)
test_embed = tokenize.texts_to_sequences(name_test)

max_seq_length = 170
train_embed = keras.preprocessing.sequence.pad_sequences(
    train_embed, maxlen=max_seq_length, padding="post")
test_embed = keras.preprocessing.sequence.pad_sequences(
    test_embed, maxlen=max_seq_length, padding="post")

In [52]:
# Define our deep model with the Functional API
deep_inputs = layers.Input(shape=(max_seq_length,))
embedding = layers.Embedding(vocab_size, 8, input_length=max_seq_length)(deep_inputs)
embedding = layers.Flatten()(embedding)
embed_out = layers.Dense(1)(embedding)
deep_model = keras.Model(inputs=deep_inputs, outputs=embed_out)
deep_model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])
plot_model(deep_model, to_file='model_plotB.png', show_shapes=True, show_layer_names=True)

Image(url = "model_plotB.png", width = 300, height = 300)

In [53]:
# Combine wide and deep into one model
merged_out = layers.concatenate([wide_model.output, deep_model.output])
merged_out = layers.Dense(1)(merged_out)
combined_modelOne = keras.Model(wide_model.input + [deep_model.input], merged_out)
combined_modelTwo = keras.Model(wide_model.input + [deep_model.input], merged_out)
plot_model(combined_modelOne, to_file='model_plotC.png', show_shapes=True, show_layer_names=True)
combined_modelOne.compile(loss='mse', optimizer='adam', metrics=['accuracy'])
combined_modelTwo.compile(loss='mse', optimizer='adam', metrics=['accuracy'])

Image(url = "model_plotC.png", width = 800, height = 800)

In [54]:
# Run training
combined_modelOne.fit([name_bow_train, category_train] + [train_embed], labels_trainOne, epochs=5000, batch_size=256)

Epoch 1/5000
131/131 [==============================] - 1s 7ms/step - loss: 18143.3594 - acc: 0.0000e+00
Epoch 2/5000
131/131 [==============================] - 0s 53us/step - loss: 18121.9395 - acc: 0.0000e+00
Epoch 3/5000
131/131 [==============================] - 0s 38us/step - loss: 18103.5859 - acc: 0.0000e+00
Epoch 4/5000
131/131 [==============================] - 0s 45us/step - loss: 18084.0195 - acc: 0.0000e+00
Epoch 5/5000
131/131 [==============================] - 0s 46us/step - loss: 18063.7090 - acc: 0.0000e+00
Epoch 6/5000
131/131 [==============================] - 0s 38us/step - loss: 18042.7676 - acc: 0.0000e+00
Epoch 7/5000
131/131 [==============================] - 0s 38us/step - loss: 18021.1602 - acc: 0.0000e+00
Epoch 8/5000
131/131 [==============================] - 0s 69us/step - loss: 17998.4512 - acc: 0.0000e+00
Epoch 9/5000
131/131 [==============================] - 0s 45us/step - loss: 17975.5625 - acc: 0.0000e+00
Epoch 10/5000
131/131 [========================

Epoch 78/5000
131/131 [==============================] - 0s 46us/step - loss: 13538.8691 - acc: 0.0000e+00
Epoch 79/5000
131/131 [==============================] - 0s 47us/step - loss: 13443.2637 - acc: 0.0000e+00
Epoch 80/5000
131/131 [==============================] - 0s 46us/step - loss: 13347.4395 - acc: 0.0000e+00
Epoch 81/5000
131/131 [==============================] - 0s 30us/step - loss: 13252.0625 - acc: 0.0000e+00
Epoch 82/5000
131/131 [==============================] - 0s 38us/step - loss: 13157.2119 - acc: 0.0000e+00
Epoch 83/5000
131/131 [==============================] - 0s 38us/step - loss: 13063.5000 - acc: 0.0000e+00
Epoch 84/5000
131/131 [==============================] - 0s 40us/step - loss: 12969.9365 - acc: 0.0000e+00
Epoch 85/5000
131/131 [==============================] - 0s 38us/step - loss: 12877.6582 - acc: 0.0000e+00
Epoch 86/5000
131/131 [==============================] - 0s 37us/step - loss: 12785.7031 - acc: 0.0000e+00
Epoch 87/5000
131/131 [==============

Epoch 155/5000
131/131 [==============================] - 0s 38us/step - loss: 10113.8906 - acc: 0.0000e+00
Epoch 156/5000
131/131 [==============================] - 0s 53us/step - loss: 10108.6172 - acc: 0.0000e+00
Epoch 157/5000
131/131 [==============================] - 0s 61us/step - loss: 10103.4082 - acc: 0.0000e+00
Epoch 158/5000
131/131 [==============================] - 0s 61us/step - loss: 10098.2695 - acc: 0.0000e+00
Epoch 159/5000
131/131 [==============================] - 0s 46us/step - loss: 10093.1953 - acc: 0.0000e+00
Epoch 160/5000
131/131 [==============================] - 0s 46us/step - loss: 10088.1758 - acc: 0.0000e+00
Epoch 161/5000
131/131 [==============================] - 0s 61us/step - loss: 10083.2080 - acc: 0.0000e+00
Epoch 162/5000
131/131 [==============================] - 0s 69us/step - loss: 10078.3047 - acc: 0.0000e+00
Epoch 163/5000
131/131 [==============================] - 0s 60us/step - loss: 10073.4287 - acc: 0.0000e+00
Epoch 164/5000
131/131 [====

Epoch 232/5000
131/131 [==============================] - 0s 53us/step - loss: 9791.2822 - acc: 0.0000e+00
Epoch 233/5000
131/131 [==============================] - 0s 45us/step - loss: 9787.6904 - acc: 0.0000e+00
Epoch 234/5000
131/131 [==============================] - 0s 36us/step - loss: 9784.1074 - acc: 0.0000e+00
Epoch 235/5000
131/131 [==============================] - 0s 46us/step - loss: 9780.5400 - acc: 0.0000e+00
Epoch 236/5000
131/131 [==============================] - 0s 46us/step - loss: 9776.9766 - acc: 0.0000e+00
Epoch 237/5000
131/131 [==============================] - 0s 46us/step - loss: 9773.4150 - acc: 0.0000e+00
Epoch 238/5000
131/131 [==============================] - 0s 46us/step - loss: 9769.8525 - acc: 0.0000e+00
Epoch 239/5000
131/131 [==============================] - 0s 46us/step - loss: 9766.2988 - acc: 0.0000e+00
Epoch 240/5000
131/131 [==============================] - 0s 46us/step - loss: 9762.7207 - acc: 0.0000e+00
Epoch 241/5000
131/131 [=============

Epoch 309/5000
131/131 [==============================] - 0s 53us/step - loss: 9522.7305 - acc: 0.0000e+00
Epoch 310/5000
131/131 [==============================] - 0s 42us/step - loss: 9519.2441 - acc: 0.0000e+00
Epoch 311/5000
131/131 [==============================] - 0s 45us/step - loss: 9515.7168 - acc: 0.0000e+00
Epoch 312/5000
131/131 [==============================] - 0s 60us/step - loss: 9512.1875 - acc: 0.0000e+00
Epoch 313/5000
131/131 [==============================] - 0s 68us/step - loss: 9508.6797 - acc: 0.0000e+00
Epoch 314/5000
131/131 [==============================] - 0s 46us/step - loss: 9505.1396 - acc: 0.0000e+00
Epoch 315/5000
131/131 [==============================] - 0s 46us/step - loss: 9501.5869 - acc: 0.0000e+00
Epoch 316/5000
131/131 [==============================] - 0s 46us/step - loss: 9498.0146 - acc: 0.0000e+00
Epoch 317/5000
131/131 [==============================] - 0s 69us/step - loss: 9494.4336 - acc: 0.0000e+00
Epoch 318/5000
131/131 [=============

...

Epoch 4650/5000
131/131 [==============================] - 0s 38us/step - loss: 3624.3762 - acc: 0.0000e+00
Epoch 4651/5000
131/131 [==============================] - 0s 46us/step - loss: 3624.3748 - acc: 0.0000e+00
Epoch 4652/5000
131/131 [==============================] - 0s 38us/step - loss: 3624.3730 - acc: 0.0000e+00
Epoch 4653/5000
131/131 [==============================] - 0s 30us/step - loss: 3624.3713 - acc: 0.0000e+00
Epoch 4654/5000
131/131 [==============================] - 0s 30us/step - loss: 3624.3701 - acc: 0.0000e+00
Epoch 4655/5000
131/131 [==============================] - 0s 30us/step - loss: 3624.3689 - acc: 0.0000e+00
Epoch 4656/5000
131/131 [==============================] - 0s 53us/step - loss: 3624.3672 - acc: 0.0000e+00
Epoch 4657/5000
131/131 [==============================] - 0s 55us/step - loss: 3624.3657 - acc: 0.0000e+00
Epoch 4658/5000
131/131 [==============================] - 0s 45us/step - loss: 3624.3640 - acc: 0.0000e+00
Epoch 4659/5000
131/131 [===

Epoch 4726/5000
131/131 [==============================] - 0s 38us/step - loss: 3624.2786 - acc: 0.0000e+00
Epoch 4727/5000
131/131 [==============================] - 0s 61us/step - loss: 3624.2776 - acc: 0.0000e+00
Epoch 4728/5000
131/131 [==============================] - 0s 38us/step - loss: 3624.2764 - acc: 0.0000e+00
Epoch 4729/5000
131/131 [==============================] - 0s 38us/step - loss: 3624.2759 - acc: 0.0000e+00
Epoch 4730/5000
131/131 [==============================] - 0s 51us/step - loss: 3624.2747 - acc: 0.0000e+00
Epoch 4731/5000
131/131 [==============================] - 0s 53us/step - loss: 3624.2737 - acc: 0.0000e+00
Epoch 4732/5000
131/131 [==============================] - 0s 46us/step - loss: 3624.2725 - acc: 0.0000e+00
Epoch 4733/5000
131/131 [==============================] - 0s 61us/step - loss: 3624.2715 - acc: 0.0000e+00
Epoch 4734/5000
131/131 [==============================] - 0s 53us/step - loss: 3624.2705 - acc: 0.0000e+00
Epoch 4735/5000
131/131 [===

Epoch 4802/5000
131/131 [==============================] - 0s 46us/step - loss: 3624.2131 - acc: 0.0000e+00
Epoch 4803/5000
131/131 [==============================] - 0s 53us/step - loss: 3624.2124 - acc: 0.0000e+00
Epoch 4804/5000
131/131 [==============================] - 0s 38us/step - loss: 3624.2117 - acc: 0.0000e+00
Epoch 4805/5000
131/131 [==============================] - 0s 30us/step - loss: 3624.2109 - acc: 0.0000e+00
Epoch 4806/5000
131/131 [==============================] - 0s 61us/step - loss: 3624.2104 - acc: 0.0000e+00
Epoch 4807/5000
131/131 [==============================] - 0s 46us/step - loss: 3624.2100 - acc: 0.0000e+00
Epoch 4808/5000
131/131 [==============================] - 0s 53us/step - loss: 3624.2090 - acc: 0.0000e+00
Epoch 4809/5000
131/131 [==============================] - 0s 61us/step - loss: 3624.2083 - acc: 0.0000e+00
Epoch 4810/5000
131/131 [==============================] - 0s 76us/step - loss: 3624.2078 - acc: 0.0000e+00
Epoch 4811/5000
131/131 [===

Epoch 4878/5000
131/131 [==============================] - 0s 53us/step - loss: 3624.1697 - acc: 0.0000e+00
Epoch 4879/5000
131/131 [==============================] - 0s 38us/step - loss: 3624.1692 - acc: 0.0000e+00
Epoch 4880/5000
131/131 [==============================] - 0s 38us/step - loss: 3624.1685 - acc: 0.0000e+00
Epoch 4881/5000
131/131 [==============================] - 0s 59us/step - loss: 3624.1682 - acc: 0.0000e+00
Epoch 4882/5000
131/131 [==============================] - 0s 43us/step - loss: 3624.1682 - acc: 0.0000e+00
Epoch 4883/5000
131/131 [==============================] - 0s 59us/step - loss: 3624.1675 - acc: 0.0000e+00
Epoch 4884/5000
131/131 [==============================] - 0s 38us/step - loss: 3624.1670 - acc: 0.0000e+00
Epoch 4885/5000
131/131 [==============================] - 0s 68us/step - loss: 3624.1667 - acc: 0.0000e+00
Epoch 4886/5000
131/131 [==============================] - 0s 61us/step - loss: 3624.1663 - acc: 0.0000e+00
Epoch 4887/5000
131/131 [===

Epoch 4954/5000
131/131 [==============================] - 0s 38us/step - loss: 3624.1421 - acc: 0.0000e+00
Epoch 4955/5000
131/131 [==============================] - 0s 68us/step - loss: 3624.1416 - acc: 0.0000e+00
Epoch 4956/5000
131/131 [==============================] - 0s 38us/step - loss: 3624.1414 - acc: 0.0000e+00
Epoch 4957/5000
131/131 [==============================] - 0s 46us/step - loss: 3624.1411 - acc: 0.0000e+00
Epoch 4958/5000
131/131 [==============================] - 0s 45us/step - loss: 3624.1409 - acc: 0.0000e+00
Epoch 4959/5000
131/131 [==============================] - 0s 61us/step - loss: 3624.1406 - acc: 0.0000e+00
Epoch 4960/5000
131/131 [==============================] - 0s 46us/step - loss: 3624.1401 - acc: 0.0000e+00
Epoch 4961/5000
131/131 [==============================] - 0s 69us/step - loss: 3624.1399 - acc: 0.0000e+00
Epoch 4962/5000
131/131 [==============================] - 0s 38us/step - loss: 3624.1399 - acc: 0.0000e+00
Epoch 4963/5000
131/131 [===

In [55]:
combined_modelOne.evaluate([name_bow_test, category_test] + [test_embed], labels_testOne, batch_size=256)

131/131 [==============================] - 0s 2ms/step


[3624.130126953125, 0.0]

In [56]:
# Generate predictions
predictions = combined_modelOne.predict([name_bow_test, category_test] + [test_embed])

In [57]:
# Compare predictions with actual values for the first few items in our test dataset
num_predictions = 30
diff = 0
num_correct = 0
relative_tolerance = 0

for i in range(num_predictions):
    val = predictions[i]
    print(name_test.iloc[i])
    print('Predicted Price: ', val[0], 'Actual Price: ', labels_testOne.iloc[i], '\n')
    diff += abs(val[0] - labels_testOne.iloc[i])
    relative_tolarance = (labels_testOne.iloc[i] / 10)
    if(abs(val[0] - labels_testOne.iloc[i]) <= relative_tolarance):
        num_correct += 1

Bowmore 15 Year Old Single Malt Scotch
Predicted Price:  89.664185 Actual Price:  70.0 

Pappy Van Winkle 20 year old Family Reserve Kentucky Straight Bourbon Whiskey
Predicted Price:  72.99938 Actual Price:  93.0 

Sazerac Kentucky Straight Rye Whiskey
Predicted Price:  54.666306 Actual Price:  27.0 

Old Pulteney 21 Year Old Single Malt Scotch Whisky
Predicted Price:  121.03928 Actual Price:  120.0 

Wild Turkey Bourbon 101
Predicted Price:  51.422592 Actual Price:  20.0 

Pappy Van Winkle 15 Year Old Family Reserve Whiskey
Predicted Price:  57.998543 Actual Price:  58.0 

Jim Beam Black Bourbon
Predicted Price:  51.422592 Actual Price:  20.0 

Highland Park 25-Year-Old Orkney Islands Single Malt Whisky
Predicted Price:  239.98845 Actual Price:  240.0 

Old Forester Straight Bourbon 86 Proof
Predicted Price:  19.092133 Actual Price:  19.0 

Dalmore 18 Year Old Single Highland Malt Scotch Whisky
Predicted Price:  163.83316 Actual Price:  165.0 

Cardhu 12 Year Old Single Malt Scotch
P

In [58]:
# Compare the average difference between actual price and the model's predicted price
print('Average prediction difference: ', diff / num_predictions)

# Print the accuracy
correct_percentage = (num_correct/num_predictions) * 100
print("Correct rate:", correct_percentage, "%")

Average prediction difference:  33.42488403320313
Correct rate: 33.33333333333333 %


In [59]:
# Run training
combined_modelTwo.fit([name_bow_train, category_train] + [train_embed], labels_trainTwo, epochs=5000, batch_size=256)

Epoch 1/5000
131/131 [==============================] - 1s 6ms/step - loss: 6642.3604 - acc: 0.0000e+00
Epoch 2/5000
131/131 [==============================] - 0s 45us/step - loss: 6538.4448 - acc: 0.0000e+00
Epoch 3/5000
131/131 [==============================] - 0s 45us/step - loss: 6450.0103 - acc: 0.0000e+00
Epoch 4/5000
131/131 [==============================] - 0s 38us/step - loss: 6359.5747 - acc: 0.0000e+00
Epoch 5/5000
131/131 [==============================] - 0s 46us/step - loss: 6270.4844 - acc: 0.0000e+00
Epoch 6/5000
131/131 [==============================] - 0s 38us/step - loss: 6184.2202 - acc: 0.0000e+00
Epoch 7/5000
131/131 [==============================] - 0s 53us/step - loss: 6098.5625 - acc: 0.0000e+00
Epoch 8/5000
131/131 [==============================] - 0s 45us/step - loss: 6018.5688 - acc: 0.0000e+00
Epoch 9/5000
131/131 [==============================] - 0s 46us/step - loss: 5940.6797 - acc: 0.0000e+00
Epoch 10/5000
131/131 [==============================] -

Epoch 79/5000
131/131 [==============================] - 0s 46us/step - loss: 3332.0269 - acc: 0.0000e+00
Epoch 80/5000
131/131 [==============================] - 0s 44us/step - loss: 3309.0745 - acc: 0.0000e+00
Epoch 81/5000
131/131 [==============================] - 0s 46us/step - loss: 3286.4609 - acc: 0.0000e+00
Epoch 82/5000
131/131 [==============================] - 0s 53us/step - loss: 3263.9163 - acc: 0.0000e+00
Epoch 83/5000
131/131 [==============================] - 0s 45us/step - loss: 3241.6089 - acc: 0.0000e+00
Epoch 84/5000
131/131 [==============================] - 0s 46us/step - loss: 3219.5137 - acc: 0.0000e+00
Epoch 85/5000
131/131 [==============================] - 0s 53us/step - loss: 3197.7434 - acc: 0.0000e+00
Epoch 86/5000
131/131 [==============================] - 0s 45us/step - loss: 3176.1838 - acc: 0.0000e+00
Epoch 87/5000
131/131 [==============================] - 0s 46us/step - loss: 3154.7644 - acc: 0.0000e+00
Epoch 88/5000
131/131 [=======================

Epoch 156/5000
131/131 [==============================] - 0s 38us/step - loss: 2014.5985 - acc: 0.0000e+00
Epoch 157/5000
131/131 [==============================] - 0s 46us/step - loss: 2002.0153 - acc: 0.0000e+00
Epoch 158/5000
131/131 [==============================] - 0s 38us/step - loss: 1989.5229 - acc: 0.0000e+00
Epoch 159/5000
131/131 [==============================] - 0s 46us/step - loss: 1977.2006 - acc: 0.0000e+00
Epoch 160/5000
131/131 [==============================] - 0s 38us/step - loss: 1965.0043 - acc: 0.0000e+00
Epoch 161/5000
131/131 [==============================] - 0s 31us/step - loss: 1952.8660 - acc: 0.0000e+00
Epoch 162/5000
131/131 [==============================] - 0s 52us/step - loss: 1940.8416 - acc: 0.0000e+00
Epoch 163/5000
131/131 [==============================] - 0s 30us/step - loss: 1928.8732 - acc: 0.0000e+00
Epoch 164/5000
131/131 [==============================] - 0s 30us/step - loss: 1916.9849 - acc: 0.0000e+00
Epoch 165/5000
131/131 [=============

Epoch 233/5000
131/131 [==============================] - 0s 53us/step - loss: 1283.0533 - acc: 0.0000e+00
Epoch 234/5000
131/131 [==============================] - 0s 46us/step - loss: 1276.1737 - acc: 0.0000e+00
Epoch 235/5000
131/131 [==============================] - 0s 38us/step - loss: 1269.3912 - acc: 0.0000e+00
Epoch 236/5000
131/131 [==============================] - 0s 46us/step - loss: 1262.6572 - acc: 0.0000e+00
Epoch 237/5000
131/131 [==============================] - 0s 49us/step - loss: 1255.9600 - acc: 0.0000e+00
Epoch 238/5000
131/131 [==============================] - 0s 45us/step - loss: 1249.3199 - acc: 0.0000e+00
Epoch 239/5000
131/131 [==============================] - 0s 53us/step - loss: 1242.7157 - acc: 0.0000e+00
Epoch 240/5000
131/131 [==============================] - 0s 61us/step - loss: 1236.1505 - acc: 0.0000e+00
Epoch 241/5000
131/131 [==============================] - 0s 53us/step - loss: 1229.6500 - acc: 0.0000e+00
Epoch 242/5000
131/131 [=============

Epoch 310/5000
131/131 [==============================] - 0s 53us/step - loss: 882.1846 - acc: 0.0000e+00
Epoch 311/5000
131/131 [==============================] - 0s 61us/step - loss: 878.2817 - acc: 0.0000e+00
Epoch 312/5000
131/131 [==============================] - 0s 54us/step - loss: 874.3992 - acc: 0.0000e+00
Epoch 313/5000
131/131 [==============================] - 0s 61us/step - loss: 870.5842 - acc: 0.0000e+00
Epoch 314/5000
131/131 [==============================] - 0s 46us/step - loss: 866.8150 - acc: 0.0000e+00
Epoch 315/5000
131/131 [==============================] - 0s 53us/step - loss: 863.0712 - acc: 0.0000e+00
Epoch 316/5000
131/131 [==============================] - 0s 68us/step - loss: 859.3372 - acc: 0.0000e+00
Epoch 317/5000
131/131 [==============================] - 0s 45us/step - loss: 855.6237 - acc: 0.0000e+00
Epoch 318/5000
131/131 [==============================] - 0s 54us/step - loss: 851.9284 - acc: 0.0000e+00
Epoch 319/5000
131/131 [======================

...

Epoch 4676/5000
131/131 [==============================] - 0s 53us/step - loss: 28.0939 - acc: 0.0000e+00
Epoch 4677/5000
131/131 [==============================] - 0s 38us/step - loss: 28.0949 - acc: 0.0000e+00
Epoch 4678/5000
131/131 [==============================] - 0s 61us/step - loss: 28.0944 - acc: 0.0000e+00
Epoch 4679/5000
131/131 [==============================] - 0s 46us/step - loss: 28.0923 - acc: 0.0000e+00
Epoch 4680/5000
131/131 [==============================] - 0s 69us/step - loss: 28.0897 - acc: 0.0000e+00
Epoch 4681/5000
131/131 [==============================] - 0s 51us/step - loss: 28.0875 - acc: 0.0000e+00
Epoch 4682/5000
131/131 [==============================] - 0s 53us/step - loss: 28.0864 - acc: 0.0000e+00
Epoch 4683/5000
131/131 [==============================] - 0s 53us/step - loss: 28.0865 - acc: 0.0000e+00
Epoch 4684/5000
131/131 [==============================] - 0s 50us/step - loss: 28.0871 - acc: 0.0000e+00
Epoch 4685/5000
131/131 [=====================

Epoch 4754/5000
131/131 [==============================] - 0s 46us/step - loss: 28.0556 - acc: 0.0000e+00
Epoch 4755/5000
131/131 [==============================] - 0s 46us/step - loss: 28.0552 - acc: 0.0000e+00
Epoch 4756/5000
131/131 [==============================] - 0s 37us/step - loss: 28.0548 - acc: 0.0000e+00
Epoch 4757/5000
131/131 [==============================] - 0s 38us/step - loss: 28.0544 - acc: 0.0000e+00
Epoch 4758/5000
131/131 [==============================] - 0s 38us/step - loss: 28.0540 - acc: 0.0000e+00
Epoch 4759/5000
131/131 [==============================] - 0s 31us/step - loss: 28.0536 - acc: 0.0000e+00
Epoch 4760/5000
131/131 [==============================] - 0s 38us/step - loss: 28.0532 - acc: 0.0000e+00
Epoch 4761/5000
131/131 [==============================] - 0s 38us/step - loss: 28.0528 - acc: 0.0000e+00
Epoch 4762/5000
131/131 [==============================] - 0s 45us/step - loss: 28.0523 - acc: 0.0000e+00
Epoch 4763/5000
131/131 [=====================

Epoch 4832/5000
131/131 [==============================] - 0s 61us/step - loss: 28.0253 - acc: 0.0000e+00
Epoch 4833/5000
131/131 [==============================] - 0s 47us/step - loss: 28.0252 - acc: 0.0000e+00
Epoch 4834/5000
131/131 [==============================] - 0s 53us/step - loss: 28.0251 - acc: 0.0000e+00
Epoch 4835/5000
131/131 [==============================] - 0s 46us/step - loss: 28.0249 - acc: 0.0000e+00
Epoch 4836/5000
131/131 [==============================] - 0s 59us/step - loss: 28.0246 - acc: 0.0000e+00
Epoch 4837/5000
131/131 [==============================] - 0s 44us/step - loss: 28.0242 - acc: 0.0000e+00
Epoch 4838/5000
131/131 [==============================] - 0s 60us/step - loss: 28.0236 - acc: 0.0000e+00
Epoch 4839/5000
131/131 [==============================] - 0s 61us/step - loss: 28.0230 - acc: 0.0000e+00
Epoch 4840/5000
131/131 [==============================] - 0s 45us/step - loss: 28.0225 - acc: 0.0000e+00
Epoch 4841/5000
131/131 [=====================

Epoch 4910/5000
131/131 [==============================] - 0s 61us/step - loss: 27.9980 - acc: 0.0000e+00
Epoch 4911/5000
131/131 [==============================] - 0s 65us/step - loss: 27.9977 - acc: 0.0000e+00
Epoch 4912/5000
131/131 [==============================] - 0s 44us/step - loss: 27.9973 - acc: 0.0000e+00
Epoch 4913/5000
131/131 [==============================] - 0s 54us/step - loss: 27.9970 - acc: 0.0000e+00
Epoch 4914/5000
131/131 [==============================] - 0s 68us/step - loss: 27.9967 - acc: 0.0000e+00
Epoch 4915/5000
131/131 [==============================] - 0s 53us/step - loss: 27.9963 - acc: 0.0000e+00
Epoch 4916/5000
131/131 [==============================] - 0s 91us/step - loss: 27.9960 - acc: 0.0000e+00
Epoch 4917/5000
131/131 [==============================] - 0s 53us/step - loss: 27.9957 - acc: 0.0000e+00
Epoch 4918/5000
131/131 [==============================] - 0s 41us/step - loss: 27.9954 - acc: 0.0000e+00
Epoch 4919/5000
131/131 [=====================

Epoch 4988/5000
131/131 [==============================] - 0s 53us/step - loss: 27.9736 - acc: 0.0000e+00
Epoch 4989/5000
131/131 [==============================] - 0s 69us/step - loss: 27.9734 - acc: 0.0000e+00
Epoch 4990/5000
131/131 [==============================] - 0s 59us/step - loss: 27.9730 - acc: 0.0000e+00
Epoch 4991/5000
131/131 [==============================] - 0s 61us/step - loss: 27.9727 - acc: 0.0000e+00
Epoch 4992/5000
131/131 [==============================] - 0s 61us/step - loss: 27.9723 - acc: 0.0000e+00
Epoch 4993/5000
131/131 [==============================] - 0s 61us/step - loss: 27.9720 - acc: 0.0000e+00
Epoch 4994/5000
131/131 [==============================] - 0s 61us/step - loss: 27.9717 - acc: 0.0000e+00
Epoch 4995/5000
131/131 [==============================] - 0s 68us/step - loss: 27.9715 - acc: 0.0000e+00
Epoch 4996/5000
131/131 [==============================] - 0s 59us/step - loss: 27.9712 - acc: 0.0000e+00
Epoch 4997/5000
131/131 [=====================

In [60]:
combined_modelTwo.evaluate([name_bow_test, category_test] + [test_embed], labels_testTwo, batch_size=256)

131/131 [==============================] - 0s 2ms/step


[27.969655990600586, 0.0]

In [61]:
# Generate predictions
predictions = combined_modelTwo.predict([name_bow_test, category_test] + [test_embed])

In [62]:
# Compare predictions with actual values for the first few items in our test dataset
num_predictions = 30
diff = 0
num_correct = 0

for i in range(num_predictions):
    val = predictions[i]
    print(name_test.iloc[i])
    print('Predicted Rating: ', val[0], 'Actual Rating: ', labels_testTwo.iloc[i], '\n')
    diff += abs(val[0] - labels_testTwo.iloc[i])
    if(abs(val[0] - labels_testTwo.iloc[i]) <= 10):
        num_correct += 1

Bowmore 15 Year Old Single Malt Scotch
Predicted Rating:  98.993256 Actual Rating:  99.0 

Pappy Van Winkle 20 year old Family Reserve Kentucky Straight Bourbon Whiskey
Predicted Rating:  87.66992 Actual Rating:  94.0 

Sazerac Kentucky Straight Rye Whiskey
Predicted Rating:  87.33338 Actual Rating:  69.0 

Old Pulteney 21 Year Old Single Malt Scotch Whisky
Predicted Rating:  92.83463 Actual Rating:  95.0 

Wild Turkey Bourbon 101
Predicted Rating:  89.29004 Actual Rating:  77.0 

Pappy Van Winkle 15 Year Old Family Reserve Whiskey
Predicted Rating:  98.99156 Actual Rating:  99.0 

Jim Beam Black Bourbon
Predicted Rating:  89.29004 Actual Rating:  86.0 

Highland Park 25-Year-Old Orkney Islands Single Malt Whisky
Predicted Rating:  90.03437 Actual Rating:  90.0 

Old Forester Straight Bourbon 86 Proof
Predicted Rating:  80.966 Actual Rating:  81.0 

Dalmore 18 Year Old Single Highland Malt Scotch Whisky
Predicted Rating:  102.221924 Actual Rating:  100.0 

Cardhu 12 Year Old Single Mal

In [63]:
# Compare the average difference between actual rating and the model's predicted rating
print('Average prediction difference: ', diff / num_predictions)

# Print the accuracy
correct_percentage = (num_correct/num_predictions) * 100
print("Correct rate:", correct_percentage, "%")

Average prediction difference:  5.397881444295248
Correct rate: 90.0 %
